In [5]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 32.6 MB/s eta 0:00:0031m32.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-04-20 19:37:15.404357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 19:37:15.412177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745159835.421162  879995 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745159835.423817  879995 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745159835.430824  879995 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Num GPUs Available:  1


In [7]:
import os
import pydicom
import numpy as np
import cv2

def convert_dicom_to_png(dicom_path, output_path):
    ds = pydicom.dcmread(dicom_path)
    img = ds.pixel_array.astype(float)
    # Normalize the image to 0-255
    img = (np.maximum(img, 0) / img.max()) * 255.0
    img = np.uint8(img)
    cv2.imwrite(output_path, img)

# Convert all DICOM files in the directory
dicom_dir = 'normal_brain_data/'
output_dir = 'normal_brain_images/'
os.makedirs(output_dir, exist_ok=True)

for root, dirs, files in os.walk(dicom_dir):
    for file in files:
        if file.endswith('.dcm'):
            dicom_path = os.path.join(root, file)
            output_path = os.path.join(output_dir, file.replace('.dcm', '.png'))
            convert_dicom_to_png(dicom_path, output_path)


In [8]:
import glob

# Get file paths
tumor_images = glob.glob('tumor_data/*.png')
normal_images = glob.glob('normal_brain_images/*.png')

# Assign labels: 1 for tumor, 0 for normal
file_paths = tumor_images + normal_images
labels = [1] * len(tumor_images) + [0] * len(normal_images)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [10]:
import os
import glob
import nibabel as nib
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# -------------------
# Configuration (update paths accordingly)
# -------------------
IMG_HEIGHT, IMG_WIDTH = 300, 300
BATCH_SIZE = 8
EPOCHS_HEAD = 10
EPOCHS_FINE = 10
# Directory containing BraTS tumor cases
TUMOR_DIR = '/media/fast/New Volume/FAST/Asim/archive(1)/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
# Directory containing healthy control MRIs (same modalities as BraTS)
HEALTHY_DIR = '/path/to/Healthy_Dataset'

# -------------------
# Helper functions
# -------------------
def load_pair(img_path, mask_path):
    # Load image volume
    img_vol = nib.load(img_path).get_fdata().astype(np.float32)
    # Determine mask volume: zeros for healthy
    if mask_path is None or mask_path == 'None':
        mask_vol = np.zeros_like(img_vol, dtype=bool)
    else:
        mask_vol = nib.load(mask_path).get_fdata() > 0

    # Select central slice
    slice_idx = img_vol.shape[2] // 2
    slice_img  = img_vol[:, :, slice_idx]
    slice_mask = mask_vol[:, :, slice_idx].astype(np.float32)

    # Apply mask
    roi = slice_img * slice_mask

    # Normalize intensities to [0,1]
    if roi.max() > 0:
        roi = (roi - roi.min()) / (roi.max() - roi.min())

    # Resize to model input size
    roi_resized = cv2.resize(roi, (IMG_WIDTH, IMG_HEIGHT))

    # Stack to 3 channels for EfficientNet
    img_3ch = np.stack([roi_resized] * 3, axis=-1)
    return img_3ch

@tf.function
def tf_load_pair(img_path, mask_path, label):
    img = tf.py_function(
        func=lambda x, y: load_pair(x.decode('utf-8'), None if y.decode('utf-8')=='None' else y.decode('utf-8')),
        inp=[img_path, mask_path], Tout=tf.float32)
    img.set_shape([IMG_HEIGHT, IMG_WIDTH, 3])
    return img, label

# -------------------
# Gather file paths and labels
# -------------------
# Tumor-positive cases
tumor_imgs  = glob.glob(os.path.join(TUMOR_DIR, 'BraTS20_Training_*', '*_t1ce.nii.gz'))
tumor_masks = glob.glob(os.path.join(TUMOR_DIR, 'BraTS20_Training_*', '*_seg.nii.gz'))
# Healthy controls (if any)
healthy_imgs = glob.glob(os.path.join(HEALTHY_DIR, '*.nii.gz'))
healthy_masks = [None] * len(healthy_imgs)

# Check that we found at least tumor or healthy images
if not tumor_imgs and not healthy_imgs:
    raise ValueError(f"No images found in TUMOR_DIR ({TUMOR_DIR}) or HEALTHY_DIR ({HEALTHY_DIR}).")

# Create labels
labels_tumor   = [1] * len(tumor_imgs)
labels_healthy = [0] * len(healthy_imgs)

# Combine lists
img_list   = tumor_imgs + healthy_imgs
mask_list  = tumor_masks + healthy_masks
label_list = labels_tumor + labels_healthy

# Determine stratification: only if both classes present
unique_labels = set(label_list)
stratify_labels = label_list if len(unique_labels) > 1 else None
if stratify_labels is None:
    print("Warning: Only one class present; proceeding without stratified split.")

# Split into train/validation
X_img_train, X_img_val, X_mask_train, X_mask_val, y_train, y_val = train_test_split(
    img_list, mask_list, label_list,
    test_size=0.2,
    stratify=stratify_labels,
    random_state=42
)

# -------------------
# Build tf.data datasets
# -------------------
def make_dataset(img_paths, mask_paths, labels, batch_size, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((img_paths, mask_paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(labels), reshuffle_each_iteration=True)
    ds = ds.map(
        lambda i, m, l: tf_load_pair(i, tf.convert_to_tensor(str(m), dtype=tf.string), l),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(X_img_train, X_mask_train, y_train, BATCH_SIZE)
val_ds   = make_dataset(X_img_val,   X_mask_val,   y_val,   BATCH_SIZE, shuffle=False)

# -------------------
# Build and compile model
# -------------------
base_model = EfficientNetB3(weights='imagenet', include_top=False,
                            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, preds)
model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# -------------------
# Train classifier head
# -------------------
print('Training classification head...')
history_head = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_HEAD)

# -------------------
# Fine-tune full model
# -------------------
for layer in base_model.layers:
    layer.trainable = True
model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
print('Fine-tuning entire model...')
history_finetune = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FINE)

# -------------------
# Evaluation
# -------------------
# Gather predictions
y_true, y_pred, y_prob = [], [], []
for imgs, labs in val_ds:
    probs = model.predict(imgs)
    preds = (probs >= 0.5).astype(int).flatten()
    y_pred.extend(preds)
    y_prob.extend(probs.flatten())
    y_true.extend(labs.numpy())

# Metrics
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:\n', cm)
print('Classification Report:\n', classification_report(y_true, y_pred))
if len(set(y_true)) == 2:
    roc_auc = roc_auc_score(y_true, y_prob)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.figure(); plt.plot(fpr, tpr, label=f'AUC={roc_auc:.3f}'); plt.plot([0,1],[0,1],'--'); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve'); plt.legend(); plt.show()


ValueError: No images found in TUMOR_DIR (/media/fast/New Volume/FAST/Asim/archive(1)/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData) or HEALTHY_DIR (/path/to/Healthy_Dataset).